## DeepCTR
deepCTRの使い方

### ドキュメント
1. tensorflow  
https://deepctr-doc.readthedocs.io/en/latest/_modules/deepctr/models/xdeepfm.html
2. pytorch  
https://deepctr-torch.readthedocs.io/en/latest/

In [1]:
# !pip install deepctr

In [2]:
import pandas as pd
import numpy as np
import datetime

from sklearn.metrics import average_precision_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import KFold, GroupKFold, train_test_split
from deepctr.models import DeepFM, xDeepFM
from deepctr.feature_column import SparseFeat, DenseFeat,get_feature_names

In [ ]:
train = pd.read_csv('inputs/train.csv')
test =  pd.read_csv('inputs/test.csv')

## 評価指標の定義

In [ ]:
def metric(va_y, pred):
    score = average_precision_score(va_y, pred)
    return score

## 前処理
省略

※ dense_featuresに対してはスケーリングを行う

## データの整形

In [ ]:
sparse_features = ["カラムを指定"]
dense_features = ["カラムを指定"]

In [ ]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(), embedding_dim=4)
                       for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                      for feat in dense_features]
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

## 学習

In [ ]:
y_pred = np.zeros(len(test))
scores = []
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for i, (tr_idx, va_idx) in enumerate(kf.split(train)):
    tr, va = train.iloc[tr_idx], train.iloc[va_idx]
    train_model_input = {name:tr[name] for name in feature_names}
    va_model_input = {name:va[name] for name in feature_names}  
    model = xDeepFM(linear_feature_columns,dnn_feature_columns,task='binary')
    model.compile("adam", "binary_crossentropy",
                  metrics=['binary_crossentropy'], )

    history = model.fit(train_model_input, tr['target'].values,
                        batch_size=256, epochs=4, verbose=2, validation_split=0.0, )
    pred_ans = model.predict(va_model_input, batch_size=256)
    print(metric(va['target'], pred_ans))
    test_model_input = {name:test[name] for name in feature_names}
    pred_ans = model.predict(test_model_input, batch_size=256)
    pred_ans = pred_ans.flatten()
    y_pred += pred_ans / 4